In [461]:
import scipy.signal
import numpy as np
from pymatreader import read_mat
import os
from matplotlib.pyplot import plot
import resampy
import pandas as pd

In [462]:
# Create a list with all the dataset files
dataset_files = []
dataset_folder = 'dataset' # dataset folder to load data
for file in os.listdir(dataset_folder):
    dataset_files.append(os.path.join(dataset_folder, file)) # append relative path

In [463]:
def preprocess(file):
    mat = read_mat(file)
    bci_exp = mat['bciexp'] # reference to the bci exp data which are the only relevant data
    labels = bci_exp['label'] # all channels

    sampling_rate = bci_exp['srate']
    downsampling_fact = 5 # downsampling factor, 250 / 5 = 50 Hz
    bandpass = np.array([1, 12.5]) # cutoff frequencies for the bandpass filter
    interval_len = .75 # seconds

    # calculate bandpass filter coefficients
    butter_b, butter_a = scipy.signal.butter(N=2, Wn=bandpass / (sampling_rate / 2), btype='bandpass')

    channels_of_interest = ['O9', 'CP1', 'CP2', 'O10', 'P7', 'P3', 'Pz',
            'P4', 'P8', 'PO7', 'PO3', 'Oz', 'PO4', 'PO8']

    # get the index of each channel in labels array
    channel_indexes = np.array([labels.index(ch) for ch in channels_of_interest])
    lmast_channel_idx = np.char.equal('LMAST', labels)

    # number of samples per analysis window
    num_samples_per_window = int(interval_len * sampling_rate / downsampling_fact) - 1

    stimuli = np.array(bci_exp['stim'], dtype=np.double)
    num_stimuli = np.sum(np.diff( np.sum(stimuli[:, :, 0], axis=0)) > 0)

    eeg_data = bci_exp['data']
    num_trials = eeg_data.shape[2]
    num_channels = len(channel_indexes)
    data = np.zeros(shape=(num_channels, num_samples_per_window, num_stimuli, num_trials))
    model = np.zeros(shape=(num_samples_per_window, num_samples_per_window, num_stimuli, num_trials))

    # For each trial
    for tr in range(num_trials):
        rdat = eeg_data[channel_indexes, :, tr] - (eeg_data[lmast_channel_idx, :, tr] / 2)
        filtfilt_signal = scipy.signal.filtfilt(butter_b, butter_a, rdat, padtype='odd',
                                                padlen=3 * (max(len(butter_b),len(butter_a)) - 1))
        rdat = scipy.signal.resample_poly(filtfilt_signal, 1, downsampling_fact, axis=1)
        rdat = rdat.T
        x1 = np.insert(stimuli[0, :, tr], 0, 0, axis=0)
        x2 = np.insert(stimuli[1, :, tr], 0, 0, axis=0)
        stim = np.array((np.diff(x1) > 0) + (np.diff(x2) > 0), dtype=np.double)

        stim_onsets = np.array(np.where(stim != 0))[0]

        for st in range(num_stimuli):
            start = int(np.ceil(stim_onsets[st] / downsampling_fact))
            idx = np.arange(start, start + num_samples_per_window)

            data[:, :, st, tr] = rdat[idx, :].T
            model[:, :, st, tr] = stim[stim_onsets[st]] * np.eye(num_samples_per_window)

    return model, data


In [464]:
model, data = preprocess(dataset_files[0])
print(data.shape)

(14, 36, 10, 120)
